In [ ]:
import sys
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
sys.path.append(os.getenv("PY_MODULES_PATH"))
import logger_tool as logger

logging = logger.get_logger(name='logger_tool', log_level='INFO', log_file='app.log')
import setup_dev as dev
env = dev.get_env()
kcar = dev.get_env('kcar')
kevlarai = dev.get_env('kevlarai')
fpgs = dev.get_env('fpgs')

dev.setup_tools()
dev.setup_tools('kevlarai')
dev.setup_tools('fpgs')

from schemas_neontology import SchoolNode, CalendarYearNode, TermNode, TermBreakNode, WeekNode, DayNode, PeriodNode
from schemas_neontology import SchoolHasCalendarYear, CalendarYearBelongsToSchool, SchoolHasTerm, TermBelongsToSchool, SchoolHasTermBreak, TermBreakBelongsToSchool, SchoolHasWeek, WeekBelongsToSchool, SchoolHasDay, DayBelongsToSchool, SchoolHasPeriod, PeriodBelongsToSchool, CalendarYearHasTerm, TermBelongsToCalendarYear, CalendarYearHasTermBreak, TermBreakBelongsToCalendarYear, CalendarYearHasWeek, WeekBelongsToCalendarYear, CalendarYearHasDay, DayBelongsToCalendarYear, CalendarYearHasPeriod, PeriodBelongsToCalendarYear, TermHasWeek, WeekBelongsToTerm, TermHasDay, DayBelongsToTerm, TermHasPeriod, PeriodBelongsToTerm, TermBreakHasWeek, WeekBelongsToTermBreak, TermBreakHasDay, DayBelongsToTermBreak, WeekHasDay, DayBelongsToWeek, WeekHasPeriod, PeriodBelongsToWeek, DayHasPeriod, PeriodBelongsToDay
from schemas_neontology import CalendarYearHasNextCalendarYear, CalendarYearHasPreviousCalendarYear, TermHasNextTerm, TermHasPreviousTerm, TermBreakHasNextTermBreak, TermBreakHasPreviousTermBreak, TermHasNextTermBreak, TermHasPreviousTermBreak, TermBreakHasNextTerm, TermBreakHasPreviousTerm, WeekHasNextWeek, WeekHasPreviousWeek, DayHasNextDay, DayHasPreviousDay, PeriodHasNextPeriod, PeriodHasPreviousPeriod
from schemas import TopicNode, TopicLessonNode, LearningStatementNode
from schemas import SubjectForKeyStage, KeyStageIncludesTopic, SubjectIncludesTopic, TopicIncludesTopicLesson, TopicIncludesLearningStatement, LearningStatementPartOfTopicLesson

import schema_tools as schema_tools
import get_planner as planner
import neontology_tools as neon
import neo4j_driver_tools as neo4j

import pandas as pd
from pydantic import ValidationError, BaseModel
from neontology import BaseNode, BaseRelationship
from typing import Optional, ClassVar, List # Delete after removing dummy code
from datetime import datetime, timedelta, date, time

In [ ]:
neon.init_neo4j_connection()
neo4j_driver = neo4j.get_neo4j_driver()

planner = planner.get_excel_sheets('planner.xlsx ')


In [ ]:
timetable_df = planner['simstimetablelookup_df']

def extract_sims_timetable_data(df):
    # Create a dictionary to hold the timetable data
    timetable_data = {}
    
    # Iterate over each row in the dataframe to find timetable data
    for index, row in df.iterrows():
        logging.debug(f'Processing row {index}')
        # Create a dictionary to hold the timetable data for this row
        row_data = {}
        # Extract the period identifier
        timetabled_period_identifier = row['Period']
        logging.pedantic(f'Period identifier: {timetabled_period_identifier}')
        # Check that the identifier is not none
        if timetabled_period_identifier is not None:
            # Extract the period type, 3 letter day, trim the colon get the period number
            period_type = str(timetabled_period_identifier)[0:1]
            day = str(timetabled_period_identifier)[1:4]
            period_number = str(timetabled_period_identifier)[5:].replace(':', '')
            # Create the period name
            period_name = period_type + day + period_number
        staff_code = row['Staff Code']
        staff_name = row['Teacher']
        room = row['Room']
        subject = row['Subject']
        subject_code = row['Subject code']
        class_code = row['Class']
        class_schweme_name = row['Scheme Name']

        # Add the data to the row data dictionary
        row_data['period_type'] = period_type
        row_data['day'] = day
        row_data['period_number'] = period_number
        row_data['period_name'] = period_name
        row_data['staff_code'] = staff_code
        row_data['staff_name'] = staff_name
        row_data['room'] = room
        row_data['subject'] = subject
        row_data['subject_code'] = subject_code
        row_data['class_code'] = class_code
        row_data['class_schweme_name'] = class_schweme_name

        # Add the row data to the timetable data dictionary
        timetable_data[period_name] = row_data

    return timetable_data

timetable_data = extract_sims_timetable_data(timetable_df)

logging.info(f'Timetable data: {timetable_data}')